# Introduction
I use this notebook to run manual administration tasks on the datastore.

## Setup MongoDB Environment

In [1]:
# imports
import os
from dotenv import load_dotenv
from pymongo import MongoClient

In [2]:
from training.ebisu.stats import Stats
from utils.data_structure_utils import get_nested_iterable_values

In [3]:
# initialize environment
load_dotenv('../.env')
client = MongoClient(os.environ['MONGODB_URI'])

In [4]:
db = client.chinese
lexicon_connection = db['lexicon']
vocabulary_connection = db['vocabulary']

## Pipeline to Join and Export Vocab and Lexemes

In [9]:
all_vocabulary_pipeline = [
    {
        '$lookup': {
            'from': 'lexicon', 
            'localField': 'lexeme_id', 
            'foreignField': '_id', 
            'as': 'lexeme'
        }
    }, {
        '$match': {
            'stats.written_to_definition': {
                '$exists': False
            }
        }
    }
]

results = list(vocabulary_connection.aggregate(all_vocabulary_pipeline))

with open('/home/alex/chinese_vocab.json', 'w') as f:
    f.write(str(results))

## Hotfix Scripts

In [132]:
all_lexeme_entries = list(lexicon_connection.find({}))
all_vocab_entries = list(vocabulary_connection.find({}))

In [133]:
lexeme_ids = list(map(lambda x: x['_id'], all_lexeme_entries))
vocab_lexeme_ids = list(map(lambda x: x['lexeme_id'], all_vocab_entries))

In [135]:
len(all_lexeme_entries)

64

In [114]:
# vocabulary_connection.drop()
# vocabulary_connection.insert_many(new_vocab_entries)